# Live Human Pose Estimation with OpenVINO™

This notebook demonstrates live pose estimation with OpenVINO, using the OpenPose [human-pose-estimation-0001](https://github.com/openvinotoolkit/open_model_zoo/tree/master/models/intel/human-pose-estimation-0001) model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/). Final part of this notebook shows live inference results from a webcam. Additionally, you can also upload a video file.

> **NOTE**: To use a webcam, you must run this Jupyter notebook on a computer with a webcam. If you run on a server, the webcam will not work. However, you can still do inference on a video in the final step.

#### Table of contents:

- [Imports](#Imports)
- [The model](#The-model)
    - [Download the model](#Download-the-model)
    - [Load the model](#Load-the-model)
- [Processing](#Processing)
    - [OpenPose Decoder](#OpenPose-Decoder)
    - [Process Results](#Process-Results)
    - [Draw Pose Overlays](#Draw-Pose-Overlays)
    - [Main Processing Function](#Main-Processing-Function)
- [Run](#Run)
    - [Run Live Pose Estimation](#Run-Live-Pose-Estimation)



In [ ]:
# %pip install -q "openvino>=2023.1.0"

## Imports
[back to top ⬆️](#Table-of-contents:)


In [ ]:
import collections
import time
from pathlib import Path

import cv2
import numpy as np
from IPython import display
from numpy.lib.stride_tricks import as_strided
import openvino as ov

from decoder import OpenPoseDecoder

# Fetch `notebook_utils` module
# import urllib.request
# urllib.request.urlretrieve(
#     url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py',
#     filename='notebook_utils.py'
# )
import notebook_utils as utils

## The model
[back to top ⬆️](#Table-of-contents:)

### Download the model
[back to top ⬆️](#Table-of-contents:)

Use the `download_file`, a function from the `notebook_utils` file. It automatically creates a directory structure and downloads the selected model.

If you want to download another model, replace the name of the model and precision in the code below. 

> **NOTE**: This may require a different pose decoder.

In [ ]:
# A directory where the model will be downloaded.
base_model_dir = Path("model")

# The name of the model from Open Model Zoo.
model_name = "human-pose-estimation-0001"
# Selected precision (FP32, FP16, FP16-INT8).
precision = "FP16-INT8"

model_path = base_model_dir / "intel" / model_name / precision / f"{model_name}.xml"

if not model_path.exists():
    model_url_dir = f"https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/{model_name}/{precision}/"
    utils.download_file(model_url_dir + model_name + '.xml', model_path.name, model_path.parent)
    utils.download_file(model_url_dir + model_name + '.bin', model_path.with_suffix('.bin').name, model_path.parent)

### Load the model
[back to top ⬆️](#Table-of-contents:)

Downloaded models are located in a fixed structure, which indicates a vendor, the name of the model and a precision.

Only a few lines of code are required to run the model. First, initialize OpenVINO Runtime. Then, read the network architecture and model weights from the `.bin` and `.xml` files to compile it for the desired device. Select device from dropdown list for running inference using OpenVINO.

In [ ]:
import ipywidgets as widgets

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
# Initialize OpenVINO Runtime
core = ov.Core()
# Read the network from a file.
model = core.read_model(model_path)
# Let the AUTO device decide where to load the model (you can use CPU, GPU as well).
compiled_model = core.compile_model(model=model, device_name=device.value, config={"PERFORMANCE_HINT": "LATENCY"})

# Get the input and output names of nodes.
input_layer = compiled_model.input(0)
output_layers = compiled_model.outputs

# Get the input size.
height, width = list(input_layer.shape)[2:]

Input layer has the name of the input node and output layers contain names of output nodes of the network. In the case of OpenPose Model, there is 1 input and 2 outputs: PAFs and keypoints heatmap.

In [ ]:
input_layer.any_name, [o.any_name for o in output_layers]

## Processing
[back to top ⬆️](#Table-of-contents:)

### OpenPose Decoder
[back to top ⬆️](#Table-of-contents:)

To transform the raw results from the neural network into pose estimations, you need OpenPose Decoder. It is provided in the [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/blob/master/demos/common/python/openvino/model_zoo/model_api/models/open_pose.py) and compatible with the `human-pose-estimation-0001` model.

If you choose a model other than `human-pose-estimation-0001` you will need another decoder (for example, `AssociativeEmbeddingDecoder`), which is available in the [demos section](https://github.com/openvinotoolkit/open_model_zoo/blob/master/demos/common/python/openvino/model_zoo/model_api/models/hpe_associative_embedding.py) of Open Model Zoo.

In [ ]:
decoder = OpenPoseDecoder()

### Process Results
[back to top ⬆️](#Table-of-contents:)

A bunch of useful functions to transform results into poses.

First, pool the heatmap. Since pooling is not available in numpy, use a simple method to do it directly with numpy. Then, use non-maximum suppression to get the keypoints from the heatmap. After that, decode poses by using the decoder. Since the input image is bigger than the network outputs, you need to multiply all pose coordinates by a scaling factor.

In [ ]:
# 2D pooling in numpy (from: https://stackoverflow.com/a/54966908/1624463)
def pool2d(A, kernel_size, stride, padding, pool_mode="max"):
    """
    2D Pooling

    Parameters:
        A: input 2D array
        kernel_size: int, the size of the window
        stride: int, the stride of the window
        padding: int, implicit zero paddings on both sides of the input
        pool_mode: string, 'max' or 'avg'
    """
    # Padding
    A = np.pad(A, padding, mode="constant")

    # Window view of A
    output_shape = (
        (A.shape[0] - kernel_size) // stride + 1,
        (A.shape[1] - kernel_size) // stride + 1,
    )
    kernel_size = (kernel_size, kernel_size)
    A_w = as_strided(
        A,
        shape=output_shape + kernel_size,
        strides=(stride * A.strides[0], stride * A.strides[1]) + A.strides
    )
    A_w = A_w.reshape(-1, *kernel_size)

    # Return the result of pooling.
    if pool_mode == "max":
        return A_w.max(axis=(1, 2)).reshape(output_shape)
    elif pool_mode == "avg":
        return A_w.mean(axis=(1, 2)).reshape(output_shape)


# non maximum suppression
def heatmap_nms(heatmaps, pooled_heatmaps):
    return heatmaps * (heatmaps == pooled_heatmaps)


# Get poses from results.
def process_results(img, pafs, heatmaps):
    # This processing comes from
    # https://github.com/openvinotoolkit/open_model_zoo/blob/master/demos/common/python/models/open_pose.py
    pooled_heatmaps = np.array(
        [[pool2d(h, kernel_size=3, stride=1, padding=1, pool_mode="max") for h in heatmaps[0]]]
    )
    nms_heatmaps = heatmap_nms(heatmaps, pooled_heatmaps)

    # Decode poses.
    poses, scores = decoder(heatmaps, nms_heatmaps, pafs)
    output_shape = list(compiled_model.output(index=0).partial_shape)
    output_scale = img.shape[1] / output_shape[3].get_length(), img.shape[0] / output_shape[2].get_length()
    # Multiply coordinates by a scaling factor.
    poses[:, :, :2] *= output_scale
    return poses, scores

## Openvino Inference

In [ ]:
image_source = "/home/gowna/openvino_notebooks/ThreePersons.jpg"
frame = cv2.imread(image_source)
input_img = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
# Create a batch of images (size = 1).
input_img = input_img.transpose((2,0,1))[np.newaxis, ...]
results = compiled_model([input_img])
pafs_output_key = compiled_model.output("Mconv7_stage2_L1")
heatmaps_output_key = compiled_model.output("Mconv7_stage2_L2")
openvino_pafs, openvino_heatmaps = results[pafs_output_key], results[heatmaps_output_key] 

## PySDK Inference

In [ ]:
import numpy as np
import degirum as dg
image_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/ThreePersons.jpg"
human_pose_det_zoo = dg.connect(dg.LOCAL, "/home/gowna/openvino_notebooks/human_pose_estimation--256x456_float_openvino_cpu_1/human_pose_estimation--256x456_float_openvino_cpu_1.json", "dg_VXc9Zfpkki4KgtjTvBdRY57QcCH6rbS3fqtw1")

In [ ]:
human_pose_det_model = human_pose_det_zoo.load_model("human_pose_estimation--256x456_float_openvino_cpu_1")
from human_pose_postprocessor import HumanPosePostprocessor
# human_pose_det_model = human_pose_det_zoo.load_model("human_pose_estimation--256x456_float_openvino_cpu_1",input_image_format="RAW", custom_postprocessor = HumanPosePostprocessor)
human_pose_det_res = human_pose_det_model(image_source)

In [ ]:
pysdk_pafs, pysdk_heatmaps = human_pose_det_res.results[0]["data"], human_pose_det_res.results[1]["data"]

## Results Comparison

In [ ]:
# openvino_pafs == pysdk_pafs
np.allclose(openvino_pafs, pysdk_pafs)

In [ ]:
openvino_heatmaps == pysdk_heatmaps
np.allclose(openvino_heatmaps, pysdk_heatmaps)